In [254]:
import numpy as np
from __future__ import division

In [265]:
%matplotlib inline
import wave, struct, numpy as np, matplotlib.mlab as mlab, pylab as pl

filename = "CML_Recording_Both.wav"
w = wave.open(filename,"rb")

#returns a named tuple (nchannels, sampwidth, framerate, 
# nframes, comptype, compname)
waveParams = w.getparams()

s = w.readframes(waveParams[3])
w.close()
waveArray = np.fromstring(s, np.int16)

spectrum, freq, bins = mlab.specgram(waveArray, NFFT=256,Fs=waveParams[2],sides='onesided')


In [266]:
spectrum.shape

(129, 11263)

In [302]:
def itakura_saito(y,x):
    y = np.array(y)
    x = np.array(x)
    return np.dot((y/x - np.log(y/x) -1) , np.ones(y.shape))
    


def get_h(eps,W,H, v):
    
    
    div = [0]* H.T.shape[0]
    for i, h in enumerate(H.T):
        div[i] = itakura_saito(eps+v, eps+np.dot(W,h))
    
    index = np.argmin(div)
    return np.array(H.T[index])



In [345]:
import random
def online_nmf(spectrum, W, H,A, B, rho, beta, n, eps):
    A_list = []
    B_list = []
   
    a = np.zeros(W.shape)
    b = np.zeros(W.shape)

    t = 1
    W_old = W
    k = W.shape[1]
    while np.linalg.norm(W - W_old, ord = "fro") < n:
        t = t+1 
        
        ind = random.randint(0, len(spectrum.T))
        v = spectrum.T[ind]
        h = get_h(eps,W,H, v)
        h = h.reshape(h.shape[0],1)
        v = v.reshape(v.shape[0],1)
        den = eps + np.dot(W, h)
        
        a += np.dot(np.dot(((eps+v)/(den**2)), h.T).T, np.dot(W,W.T)).T
        b += np.dot(1/den, h.T)
        
        if t % beta == 0:
            A = A + rho*a
            a = 0
            B = B + rho*b
            b = 0
            W_old = W
            W = np.sqrt(A/B)
            
            W = np.array([x/sum(x) for x in zip(*W)]).T
            A = np.array([x/sum(x) for x in zip(*A)]).T
            B = np.array([x*sum(x) for x in zip(*B)]).T
            
        if t > 30:
            print(W.shape)
            break

eps = 1e-12
v = spectrum.T[0]
K = 2
W = np.random.rand(spectrum.shape[0],K)
H = np.zeros((K, spectrum.shape[1]))

A = np.zeros(W.shape)
B = np.zeros(W.shape)


online_nmf(spectrum, W, H, A, B, 0.5, 100, 1e-3, eps)

(129, 2)


In [ ]:
eps = 1e-12
random.seed(12222015)

In [92]:
s = [sum(x) for x in zip(*W)]
W = [sum(x) for x in zip(*W)]

In [93]:
p.shape

(1, 129)

In [245]:
W = np.array([[1,1], [1,2], [2,2]])
X = [1,2]



In [246]:
W

array([[1, 1],
       [1, 2],
       [2, 2]])

In [237]:
X = W.T
for i in range(2):
    
    col_sum = X[i].sum()
    print(X[i]/col_sum)
    X[i] = (X[i]/col_sum)
    print(X[i])


[ 0.25  0.25  0.5 ]
[0 0 0]
[ 0.2  0.4  0.4]
[0 0 0]


In [251]:
z = np.array([x/sum(x) for x in zip(*W)])

In [252]:
z.T

array([[ 0.25,  0.2 ],
       [ 0.25,  0.4 ],
       [ 0.5 ,  0.4 ]])